In [34]:
import pandas as pd
import tensorflow as tf
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict
from pyspark.sql.types import *
from pyspark.sql import Row, SparkSession
import pyspark.sql.functions
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, ArrayType

In [35]:
spark = SparkSession.builder\
    .master("local[4]")\
    .appName("sql.functions tests")\
    .getOrCreate()
sc = spark.sparkContext

In [36]:
train_file_loc = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train_labels.csv'
test_file_loc = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test_labels.csv'

In [37]:
TRAINING_FILE_LOCATION = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/*.jpg'
TEST_FILE_LOCATION = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/*.jpg'

In [38]:
def class_text_to_int(row_label):
    if row_label == 'nine':
        return 1
    elif row_label == 'ten':
        return 2
    elif row_label == 'jack':
        return 3
    elif row_label == 'queen':
        return 4
    elif row_label == 'king':
        return 5
    elif row_label == 'ace':
        return 6
    else:
        return None

In [39]:
def load_label_files(label_filename):
    df = pd.read_csv(label_filename)
    grouped = df['filename'].unique()
    
    csv_df = pd.DataFrame()
    for files in grouped:
        mydf = df[df['filename'] == files]
        #print(mydf.head())
        height = mydf['height'].unique()[0]
        width = mydf['width'].unique()[0]
        filename = mydf.filename.unique()[0]
        label = mydf['class'].tolist()
        #print(list(label))
        num_label = mydf['class'].map(class_text_to_int).tolist()
        xmin = mydf['xmin'].map(lambda x: x/width).tolist()
        ymin = mydf['ymin'].map(lambda x: x/height).tolist()
        xmax = mydf['xmax'].map(lambda x: x/width).tolist()
        ymax = mydf['ymax'].map(lambda x: x/height).tolist()
        temp = pd.DataFrame({'image/filename':filename, 'image/width':width, 'image/height':height, 
                         'image/object/class/label':[num_label], 'image/object/bbox/xmin':[xmin], 'image/object/bbox/ymin':[ymin], 
                        'image/object/bbox/xmax':[xmax], 'image/object/bbox/ymax':[ymax], 'image/object/class/text':[label]})
        csv_df = pd.concat([csv_df, temp])
    csv_df.reset_index(inplace= True)
    csv_df.drop('index', axis=1, inplace = True)
    csv_df['image/format'] = 'jpg'
    return csv_df

In [40]:
def create_tfrecord(pandas_df, tf_record_loc):
    pand_df = sqlContext.createDataFrame(csv_df)
    image_df = spark.read.format('image').load(TRAINING_FILE_LOCATION)
    image_df = image_df.withColumn('filename', image_df.image.origin.cast("string")[145:300])
    combined_df = image_df.join(pand_df, \
                            image_df.filename == pand_df['image/filename'], \
                            'inner').drop(image_df.filename)
    combined_df = combined_df.withColumn('image/source_id', combined_df['image'].origin)
    combined_df = combined_df.withColumn('image/encoded', combined_df['image'].data)
    combined_df = combined_df.drop('image')
    combined_df.write.format('tfrecords').option('recordType', 'Example').save(tf_record_loc)

In [41]:
train_df = load_label_files(train_file_loc)
test_df = load_label_files(test_file_loc)

In [42]:
create_tfrecord(train_df, 'tf_card_train')
create_tfrecord(test_df, 'tf_card_test')

In [7]:
#col_names = ['filename', 'width', 'height', 'label', 'xmin', 'ymin', 'xmax', 'ymax']

In [18]:
#label_files = spark.read.format('csv') \
#    .option("header", "true") \
#    .option("inferSchema", 'true') \
#    .load('train_label_file.csv')

In [29]:
def create_tf_example(group, path):
    #with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        #encoded_jpg = fid.read()
    #encoded_jpg_io = io.BytesIO(encoded_jpg)
    #image = Image.open(encoded_jpg_io)
    #width, height = image.size
    height = group.height[0]
    width = group.width[0]
    encoded_jpg = group.img_data[0]
    
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        # Changed to directly get class num from column
        classes.append(row['num_class'])

In [32]:
tf_rec = spark.read.format("tfrecords").option("recordType", "Example").load('pand_tf_train.record/part-r-00000')

In [33]:
tf_rec.printSchema()

root
 |-- image/object/class/text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- image/width: long (nullable = true)
 |-- image/height: long (nullable = true)
 |-- image/object/bbox/ymax: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/object/bbox/xmin: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/filename: string (nullable = true)
 |-- image/format: string (nullable = true)
 |-- image/source_id: string (nullable = true)
 |-- image/object/class/label: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- image/encoded: string (nullable = true)
 |-- image/object/bbox/xmax: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/object/bbox/ymin: array (nullable = true)
 |    |-- element: float (containsNull = true)

